# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

Assignment submitted by: Anand Shankar

In [596]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"
styles = [dict(selector="caption", props=[("font-size", "120%"),("font-weight", "bold"),("background-color", "wheat"),("font-color","black")])]

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [597]:
# Calculate the total number of unique schools
#option 1: school_count = len(school_data)
#option 2:
school_count = len(school_data_complete['school_name'].unique())
print(f'Total School Count: {school_count}')

Total School Count: 15


In [598]:
# Calculate the total number of students
#option 1: student_count = len(student_data['student_name'].unique())
#option 2:
student_count = len(school_data_complete['student_name'].unique())
print(f'Total Student Count: {student_count}')

Total Student Count: 32715


In [599]:
# Calculate the total budget
total_budget = sum(school_data["budget"])
print(f'Total Budget: ${total_budget}')

Total Budget: $24649428


In [600]:
# Calculate the average (mean) math score
average_math_score = "{:,.2f}".format(sum(student_data['math_score']) / student_count)
print(f'Average Math Score: {average_math_score}')

Average Math Score: 94.57


In [601]:
# Calculate the average (mean) reading score
average_reading_score = "{:,.2f}".format(sum(student_data['reading_score']) / student_count)
print(f'Average Reading Score: {average_reading_score}')

Average Reading Score: 98.03


In [602]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = "{:,.2f}".format(passing_math_count / float(student_count) * 100)
print(f'Percentage of students who passed math: {passing_math_percentage}')

Percentage of students who passed math: 89.78


In [603]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage =  "{:,.2f}".format(passing_reading_count / float(student_count) * 100)
print(f'Percentage of students who passed reading: {passing_reading_percentage}')

Percentage of students who passed reading: 102.74


In [604]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = "{:,.2f}".format(passing_math_reading_count /  float(student_count) * 100)
print(f'Overall percentage of students that passed math and reading: {overall_passing_rate}')

Overall percentage of students that passed math and reading: 78.03


In [605]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame(school_data.groupby(["type"])[["type","size","budget"]].sum()).reset_index().rename(columns={"size": "Total Students","budget":"Total Budget"})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

#district_summary = district_summary.loc[district_summary["type"] == 'District',:]

# Display the DataFrame
district_summary.set_index("type")
district_summary.style.hide(axis='index')
district_summary.style.set_caption("District Summary").set_table_styles(styles)

,type,Total Students,Total Budget
0,Charter,"12,194","$7,301,505.00"
1,District,"26,976","$17,347,923.00"


## School Summary

In [606]:
# Use the code provided to select the school type
school_types = pd.DataFrame(school_data.groupby(["type"]).count()["School ID"]).rename(columns={"School ID" : "Count"})
school_types

,Count
type,
Charter,8
District,7


In [607]:
# Calculate the total student count
per_school_counts = pd.DataFrame(school_data_complete.groupby(["school_name"]).count()["size"]).reset_index().set_index("school_name")
total_students_count = per_school_counts.sum().values
print(f'Total Students Count: {total_students_count}')

Total Students Count: [39170]


In [608]:
# Calculate the total school budget and per capita spending
per_school_budget = pd.DataFrame(school_data_complete.groupby(["school_name"]).mean()["budget"]).reset_index().set_index("school_name")
# per_school_budget

In [609]:
per_school_capita = pd.DataFrame.merge(per_school_budget,per_school_counts, on="school_name").reset_index().set_index("school_name")
per_school_capita["Per Capita"] = (per_school_budget["budget"] / per_school_counts["size"])
# per_school_capita

In [610]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby('school_name')['math_score'].mean().reset_index(name = 'Average Math Score').set_index("school_name")
per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean().reset_index(name = 'Average Reading Score').set_index("school_name")
# per_school_math


In [611]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete.groupby('school_name')['math_score'].apply(lambda x: (x>70).sum()).reset_index(name='Count')
school_passing_math.set_index("school_name", inplace=True)
school_passing_math

,Count
school_name,
Bailey High School,3216
Cabrera High School,1664
Figueroa High School,1880
Ford High School,1801
Griffin High School,1317
Hernandez High School,3001
Holden High School,387
Huang High School,1847
Johnson High School,3040


In [612]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete.groupby('school_name')['reading_score'].apply(lambda x: (x>70).sum()).reset_index(name='Count')
school_passing_reading.set_index("school_name", inplace=True)
# school_passing_reading

In [613]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = pd.DataFrame(school_data_complete[
  (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
  ][:])
passing_math_and_reading = passing_math_and_reading.groupby('school_name')['reading_score'].apply(lambda x: (x>70).sum()).reset_index(name='Count')
passing_math_and_reading.set_index("school_name", inplace=True)
# passing_math_and_reading

In [614]:
# Use the provided code to calculate the passing rates
# per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
# per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
# overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [615]:
#Math passing rates
per_school_passing_math = school_passing_math
per_school_passing_math["Math Passing Rate"] = (per_school_passing_math["Count"] / per_school_counts["size"] * 100)
#.apply(lambda x : format(float(x),".2f"))
del per_school_passing_math["Count"]


In [616]:
#Reading passing rates
per_school_passing_reading = school_passing_reading
per_school_passing_reading["Reading Passing Rate"] = (per_school_passing_reading["Count"] / per_school_counts["size"] * 100)
#.apply(lambda x : format(float(x),".2f"))
del per_school_passing_reading["Count"]

In [617]:
#Overall Passing Rate
overall_passing_rate = passing_math_and_reading
overall_passing_rate["Overall Passing Rate"] = (overall_passing_rate["Count"] / per_school_counts["size"] * 100)
# .apply(lambda x : format(float(x),".2f"))
del overall_passing_rate["Count"]

In [618]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = per_school_capita
# pd.merge(per_school_capita, on=["school_name"])
per_school_summary = pd.merge(per_school_summary,school_data[["school_name","type"]],on=["school_name"])
per_school_summary = pd.merge(per_school_summary,per_school_math,on=["school_name"])
per_school_summary = pd.merge(per_school_summary,per_school_reading,on=["school_name"]) 
per_school_summary = pd.merge(per_school_summary,per_school_passing_math,on=["school_name"])
per_school_summary = pd.merge(per_school_summary,per_school_passing_reading,on=["school_name"])
per_school_summary = pd.merge(per_school_summary,overall_passing_rate, on=['school_name'])
per_school_summary.rename(columns={"budget":"Total School Budget"}, inplace=True)
per_school_summary.rename(columns={"Per Capita":"Per Student Budget"}, inplace=True)

#used later save before formatting 
# Create a copy of the school summary since it has the "Per Student Budget" 
school_size_df = per_school_summary.copy()

# # Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary
per_school_summary.style.set_caption("School Summary").set_table_styles(styles)

,school_name,Total School Budget,size,Per Student Budget,type,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
0,Bailey High School,"$3,124,928.00",4976,$628.00,District,77.048432,81.033963,64.630225,79.300643,52.913987
1,Cabrera High School,"$1,081,356.00",1858,$582.00,Charter,83.061895,83.975780,89.558665,93.864370,88.213132
2,Figueroa High School,"$1,884,411.00",2949,$639.00,District,76.711767,81.158020,63.750424,78.433367,51.610715
3,Ford High School,"$1,763,916.00",2739,$644.00,District,77.102592,80.746258,65.753925,77.510040,53.158087
4,Griffin High School,"$917,500.00",1468,$625.00,Charter,83.351499,83.816757,89.713896,93.392371,87.193460
5,Hernandez High School,"$3,022,020.00",4635,$652.00,District,77.289752,80.934412,64.746494,78.187702,51.801510
6,Holden High School,"$248,087.00",427,$581.00,Charter,83.803279,83.814988,90.632319,92.740047,85.948478
7,Huang High School,"$1,910,635.00",2917,$655.00,District,76.629414,81.182722,63.318478,78.813850,51.834076
8,Johnson High School,"$3,094,650.00",4761,$650.00,District,77.072464,80.966394,63.852132,78.281874,51.564797
9,Pena High School,"$585,858.00",962,$609.00,Charter,83.839917,84.044699,91.683992,92.203742,87.422037


## Highest-Performing Schools (by % Overall Passing)

In [619]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
per_school_summary = per_school_summary.sort_values(by="Overall Passing Rate",ascending=False)
per_school_summary.head(5)
per_school_summary.style.set_caption("Highest-Performing Schools (by % Overall Passing)").set_table_styles(styles)

,school_name,Total School Budget,size,Per Student Budget,type,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
1,Cabrera High School,"$1,081,356.00",1858,$582.00,Charter,83.061895,83.975780,89.558665,93.864370,88.213132
13,Wilson High School,"$1,319,574.00",2283,$578.00,Charter,83.274201,83.989488,90.932983,93.254490,87.560228
9,Pena High School,"$585,858.00",962,$609.00,Charter,83.839917,84.044699,91.683992,92.203742,87.422037
14,Wright High School,"$1,049,400.00",1800,$583.00,Charter,83.682222,83.955000,90.277778,93.444444,87.277778
4,Griffin High School,"$917,500.00",1468,$625.00,Charter,83.351499,83.816757,89.713896,93.392371,87.193460
12,Thomas High School,"$1,043,130.00",1635,$638.00,Charter,83.418349,83.848930,90.214067,92.905199,87.094801
11,Shelton High School,"$1,056,600.00",1761,$600.00,Charter,83.359455,83.725724,89.892107,92.617831,86.712095
6,Holden High School,"$248,087.00",427,$581.00,Charter,83.803279,83.814988,90.632319,92.740047,85.948478
3,Ford High School,"$1,763,916.00",2739,$644.00,District,77.102592,80.746258,65.753925,77.510040,53.158087
0,Bailey High School,"$3,124,928.00",4976,$628.00,District,77.048432,81.033963,64.630225,79.300643,52.913987


## Bottom Performing Schools (By % Overall Passing)

In [620]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
per_school_summary = per_school_summary.sort_values(by="Overall Passing Rate")
per_school_summary.head(5)
per_school_summary.style.set_caption("Bottom Performing Schools (By % Overall Passing)").set_table_styles(styles)

,school_name,Total School Budget,size,Per Student Budget,type,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
10,Rodriguez High School,"$2,547,363.00",3999,$637.00,District,76.842711,80.744686,64.066017,77.744436,51.212803
8,Johnson High School,"$3,094,650.00",4761,$650.00,District,77.072464,80.966394,63.852132,78.281874,51.564797
2,Figueroa High School,"$1,884,411.00",2949,$639.00,District,76.711767,81.158020,63.750424,78.433367,51.610715
5,Hernandez High School,"$3,022,020.00",4635,$652.00,District,77.289752,80.934412,64.746494,78.187702,51.801510
7,Huang High School,"$1,910,635.00",2917,$655.00,District,76.629414,81.182722,63.318478,78.813850,51.834076
0,Bailey High School,"$3,124,928.00",4976,$628.00,District,77.048432,81.033963,64.630225,79.300643,52.913987
3,Ford High School,"$1,763,916.00",2739,$644.00,District,77.102592,80.746258,65.753925,77.510040,53.158087
6,Holden High School,"$248,087.00",427,$581.00,Charter,83.803279,83.814988,90.632319,92.740047,85.948478
11,Shelton High School,"$1,056,600.00",1761,$600.00,Charter,83.359455,83.725724,89.892107,92.617831,86.712095
12,Thomas High School,"$1,043,130.00",1635,$638.00,Charter,83.418349,83.848930,90.214067,92.905199,87.094801


## Math Scores by Grade

In [621]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete.loc[(school_data_complete["grade"] == "9th"),["school_name","math_score"]].rename(columns={"math_score":"9th"})
tenth_graders = school_data_complete.loc[(school_data_complete["grade"] == "10th"),["school_name","math_score"]].rename(columns={"math_score":"10th"})
eleventh_graders = school_data_complete.loc[(school_data_complete["grade"] == "11th"),["school_name","math_score"]].rename(columns={"math_score":"11th"})
twelfth_graders = school_data_complete.loc[(school_data_complete["grade"] == "12th"),["school_name","math_score"]].rename(columns={"math_score":"12th"})

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(ninth_graders_scores,tenth_graders_scores,on=["school_name"])
math_scores_by_grade = pd.merge(math_scores_by_grade,eleventh_graders_scores,on=["school_name"])
math_scores_by_grade = pd.merge(math_scores_by_grade,twelfth_graders_scores, on=["school_name"])

# # Use the code to select only the `math_score`.
# ninth_grade_math_scores = ninth_graders_scores["9th_grade_math_score"]
# tenth_grader_math_scores = tenth_graders_scores["10th_grade_math_score"]
# eleventh_grader_math_scores = eleventh_graders_scores["11th_grade_math_score"]
# twelfth_grader_math_scores = twelfth_graders_scores["12th_grade_math_score"]

# # Combine each of the scores above into single DataFrame called `math_scores_by_grade`
# math_scores_by_grade = pd.merge(ninth_grade_math_scores,tenth_grader_math_scores,on=["school_name"])
# math_scores_by_grade = pd.merge(math_scores_by_grade,eleventh_grader_math_scores,on=["school_name"])
# math_scores_by_grade = pd.merge(math_scores_by_grade,twelfth_grader_math_scores, on=["school_name"])

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade
math_scores_by_grade.style.set_caption("Math Scores by Grade").set_table_styles(styles)

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [622]:
# Use the code provided to separate the data by grade
from nbformat import read


ninth_graders = school_data_complete.loc[(school_data_complete["grade"] == "9th"),["school_name","reading_score"]].rename(columns={"reading_score":"9th"})
tenth_graders = school_data_complete.loc[(school_data_complete["grade"] == "10th"),["school_name","reading_score"]].rename(columns={"reading_score":"10th"})
eleventh_graders = school_data_complete.loc[(school_data_complete["grade"] == "11th"),["school_name","reading_score"]].rename(columns={"reading_score":"11th"})
twelfth_graders = school_data_complete.loc[(school_data_complete["grade"] == "12th"),["school_name","reading_score"]].rename(columns={"reading_score":"12th"})

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
reading_scores_by_grade = pd.merge(ninth_graders_scores,tenth_graders_scores,on=["school_name"])
reading_scores_by_grade = pd.merge(reading_scores_by_grade,eleventh_graders_scores,on=["school_name"])
reading_scores_by_grade = pd.merge(reading_scores_by_grade,twelfth_graders_scores, on=["school_name"])

# Minor data wrangling
# reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None
# Display the DataFrame
reading_scores_by_grade
reading_scores_by_grade.style.set_caption("Reading Scores by Grade").set_table_styles(styles)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [623]:
# Establish the bins 
size_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [624]:
# Create a copy of the school summary since it has the "Per Student Budget" 

In [625]:
# Use `pd.cut` to categorize spending based on the bins.
school_size_df["School Size"] = pd.cut(school_size_df["Per Student Budget"], bins=size_bins, labels=labels)
school_size_df
school_size_df.style.set_caption("Spending Ranges(Binned)").set_table_styles(styles)

,school_name,Total School Budget,size,Per Student Budget,type,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate,School Size
0,Bailey High School,3124928.000000,4976,628.000000,District,77.048432,81.033963,64.630225,79.300643,52.913987,$585-630
1,Cabrera High School,1081356.000000,1858,582.000000,Charter,83.061895,83.975780,89.558665,93.864370,88.213132,<$585
2,Figueroa High School,1884411.000000,2949,639.000000,District,76.711767,81.158020,63.750424,78.433367,51.610715,$630-645
3,Ford High School,1763916.000000,2739,644.000000,District,77.102592,80.746258,65.753925,77.510040,53.158087,$630-645
4,Griffin High School,917500.000000,1468,625.000000,Charter,83.351499,83.816757,89.713896,93.392371,87.193460,$585-630
5,Hernandez High School,3022020.000000,4635,652.000000,District,77.289752,80.934412,64.746494,78.187702,51.801510,$645-680
6,Holden High School,248087.000000,427,581.000000,Charter,83.803279,83.814988,90.632319,92.740047,85.948478,<$585
7,Huang High School,1910635.000000,2917,655.000000,District,76.629414,81.182722,63.318478,78.813850,51.834076,$645-680
8,Johnson High School,3094650.000000,4761,650.000000,District,77.072464,80.966394,63.852132,78.281874,51.564797,$645-680
9,Pena High School,585858.000000,962,609.000000,Charter,83.839917,84.044699,91.683992,92.203742,87.422037,$585-630


In [626]:
#  Calculate averages for the desired columns. 
size_math_scores = pd.DataFrame(school_size_df.groupby('School Size',dropna=False).mean()["Average Math Score"])
size_reading_scores = pd.DataFrame(school_size_df.groupby(["School Size"]).mean()["Average Reading Score"])
size_passing_math = pd.DataFrame(school_size_df.groupby(["School Size"]).mean() ["Math Passing Rate"])
size_passing_reading = pd.DataFrame(school_size_df.groupby(["School Size"]).mean()["Reading Passing Rate"])
overall_passing_spending = pd.DataFrame(school_size_df.groupby(["School Size"]).mean()["Overall Passing Rate"])

In [627]:
# Assemble into DataFrame
size_summary = pd.DataFrame(size_math_scores)
size_summary = pd.merge(size_summary,size_reading_scores,on="School Size")
size_summary = pd.merge(size_summary,size_passing_math,on="School Size")
size_summary = pd.merge(size_summary,size_passing_reading,on="School Size")
size_summary = pd.merge(size_summary,overall_passing_spending,on="School Size")

# Display results
size_summary
size_summary.style.set_caption("Scores (Binned) by School Size").set_table_styles(styles)

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
School Size,,,,,
<$585,83.455399,83.933814,90.350436,93.325838,87.249904
$585-630,81.899826,83.155286,83.980055,89.378647,78.560395
$630-645,78.518855,81.624473,70.946108,81.648261,60.769102
$645-680,76.997210,81.027843,63.972368,78.427809,51.733461


## Scores by School Size

In [628]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [629]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary["School Size"] = pd.cut(per_school_summary["size"], bins=size_bins, labels=labels)
per_school_summary
per_school_summary.style.set_caption("Scores (Binned) by School Size").set_table_styles(styles)


,school_name,Total School Budget,size,Per Student Budget,type,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate,School Size
10,Rodriguez High School,"$2,547,363.00",3999,$637.00,District,76.842711,80.744686,64.066017,77.744436,51.212803,Large (2000-5000)
8,Johnson High School,"$3,094,650.00",4761,$650.00,District,77.072464,80.966394,63.852132,78.281874,51.564797,Large (2000-5000)
2,Figueroa High School,"$1,884,411.00",2949,$639.00,District,76.711767,81.158020,63.750424,78.433367,51.610715,Large (2000-5000)
5,Hernandez High School,"$3,022,020.00",4635,$652.00,District,77.289752,80.934412,64.746494,78.187702,51.801510,Large (2000-5000)
7,Huang High School,"$1,910,635.00",2917,$655.00,District,76.629414,81.182722,63.318478,78.813850,51.834076,Large (2000-5000)
0,Bailey High School,"$3,124,928.00",4976,$628.00,District,77.048432,81.033963,64.630225,79.300643,52.913987,Large (2000-5000)
3,Ford High School,"$1,763,916.00",2739,$644.00,District,77.102592,80.746258,65.753925,77.510040,53.158087,Large (2000-5000)
6,Holden High School,"$248,087.00",427,$581.00,Charter,83.803279,83.814988,90.632319,92.740047,85.948478,Small (<1000)
11,Shelton High School,"$1,056,600.00",1761,$600.00,Charter,83.359455,83.725724,89.892107,92.617831,86.712095,Medium (1000-2000)
12,Thomas High School,"$1,043,130.00",1635,$638.00,Charter,83.418349,83.848930,90.214067,92.905199,87.094801,Medium (1000-2000)


In [630]:
# Calculate averages for the desired columns. 
size_math_scores = pd.DataFrame(per_school_summary.groupby(["School Size"]).mean()["Average Math Score"])
size_reading_scores = pd.DataFrame(per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"])
size_passing_math = pd.DataFrame(per_school_summary.groupby(["School Size"]).mean()["Math Passing Rate"])
size_passing_reading = pd.DataFrame(per_school_summary.groupby(["School Size"]).mean()["Reading Passing Rate"])
size_overall_passing = pd.DataFrame(per_school_summary.groupby(["School Size"]).mean()["Overall Passing Rate"])

In [631]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(size_math_scores)
size_summary = pd.merge(size_summary,size_reading_scores,on="School Size")
size_summary = pd.merge(size_summary,size_passing_math,on="School Size")
size_summary = pd.merge(size_summary,size_passing_reading,on="School Size")
size_summary = pd.merge(size_summary,size_overall_passing,on="School Size")

# Display results
size_summary
size_summary.style.set_caption("Scores (Binned) by School Size").set_table_styles(styles)

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,86.685258
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,87.298253
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,56.457026


## Scores by School Type

In [632]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores =  pd.DataFrame(per_school_summary.groupby(["type"]).mean()["Average Math Score"])
type_reading_scores = pd.DataFrame(per_school_summary.groupby(["type"]).mean()["Average Reading Score"])
type_passing_math = pd.DataFrame(per_school_summary.groupby(["type"]).mean()["Math Passing Rate"])
type_passing_reading = pd.DataFrame(per_school_summary.groupby(["type"]).mean()["Reading Passing Rate"])
type_overall_passing = pd.DataFrame(per_school_summary.groupby(["type"]).mean()["Overall Passing Rate"])

# Use the code provided to select new column data
average_math_score_by_type = pd.DataFrame(type_math_scores["Average Math Score"])
average_reading_score_by_type = pd.DataFrame(type_reading_scores["Average Reading Score"])
average_percent_passing_math_by_type = pd.DataFrame(type_passing_math["Math Passing Rate"])
average_percent_passing_reading_by_type = pd.DataFrame(type_passing_reading["Reading Passing Rate"])
average_percent_overall_passing_by_type = pd.DataFrame(type_overall_passing["Overall Passing Rate"])

In [633]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame(average_math_score_by_type)
type_summary = pd.merge(type_summary,average_reading_score_by_type,on="type")
type_summary = pd.merge(type_summary,average_percent_passing_math_by_type,on="type")
type_summary = pd.merge(type_summary,average_percent_passing_reading_by_type,on="type")
type_summary = pd.merge(type_summary,average_percent_overall_passing_by_type,on="type")
# Display results
type_summary
type_summary.style.set_caption("Scores (Binned) by School Size").set_table_styles(styles)

,Average Math Score,Average Reading Score,Math Passing Rate,Reading Passing Rate,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,87.177751
District,76.956733,80.966636,64.302528,78.324559,52.013711


In [637]:
print(f"---------------------------")
print(f"Summary Report")
print(f"---------------------------")
print()
print(f"From Spend Perspective...................................................... ")
print(f"From analyzing various data points from the prespective of Spending, schools that spend less have scored overall passing rate is high")
print(f"Also their average score in reading and math are higher than schools that spend more")
print(f"This does not mean spending less is effective, but surely more data is needed to understand the layers that contribute")
print(f"Further data points that could help this analysis are:")
print(f" 1. Demographic - Students and parents")
print(f" 2. School Programs to improve student experience and learnings")
print(f" 3. Skills of teachers")
print(f" 4. How is the money spent across various school programs")
print()
print(f"From Size Perspective...................................................... ")
print(f"Smaller school have performed better, this could be due to student-teacher ratio")
print(f"The difference in overall performance does not appear to be big when we compare the school size less than 1000 versus the school size between 1000-200")
print(f"Smaller schools also mean higher per student cost , so the optimal size for schools appears to be 1000-2000 students")
print()
print(f"From Type Perspective...................................................... ")
print(f"Over all charter schools appear to do better than district schools")
print(f"More data needed to break down the reason behind this")
print(f"---------------------------")

---------------------------
Summary Report
---------------------------

From Spend Perspective...................................................... 
From analyzing various data points from the prespective of Spending, schools that spend less have scored overall passing rate is high
Also their average score in reading and math are higher than schools that spend more
This does not mean spending less is effective, but surely more data is needed to understand the layers that contribute
Further data points that could help this analysis are:
 1. Demographic - Students and parents
 2. School Programs to improve student experience and learnings
 3. Skills of teachers
 4. How is the money spent across various school programs

From Size Perspective...................................................... 
Smaller school have performed better, this could be due to student-teacher ratio
The difference in overall performance does not appear to be big when we compare the school size less than 1000 ver